# Memgraph Python Intro

Here we follow this official [documentation page](https://memgraph.com/docs/client-libraries/python) from [Memgraph](https://memgraph.com/) that uses `neo4j` to connect to a Memgraph database inside a docker container.

## Pre-requisites

Before proceeding to the following executions, you need to:
1. Have [Docker](https://www.docker.com/) running locally.
2. Download the docker compose file for Memgraph and **start the service container stack**:
    1. `curl https://install.memgraph.com | sh`
    2. `docker compose up -d` inside the folder holding the compose file
3. Python dependency - `neo4j`

In [1]:
!cat compose.yaml

services:
  memgraph:
    image: memgraph/memgraph-mage:3.2
    container_name: memgraph-mage
    pull_policy: if_not_present
    ports:
      - "7687:7687"
      - "7444:7444"
    command: ["--log-level=TRACE"]

  lab:
    image: memgraph/lab:3.2.0
    container_name: memgraph-lab
    pull_policy: if_not_present
    ports:
      - "3000:3000"
    depends_on:
      - memgraph
    environment:
      - QUICK_CONNECT_MG_HOST=memgraph
      - QUICK_CONNECT_MG_PORT=7687


## Interact with a Memgraph DB using Python

In [2]:
from neo4j import GraphDatabase

In [3]:
URI = 'bolt://localhost:7687'
AUTH = ('', '')

In [4]:
with GraphDatabase.driver(URI, auth=AUTH) as client:
  client.verify_connectivity()

  # create a user in db
  records, summary, keys = client.execute_query(
    'CREATE (u:User {name: $name, password: $password}) RETURN u.name AS name;',
    name='John',
    password='pass',
    database_='memgraph',
  )
  # check result
  for record in records:
    print(record['name'])
  print(summary.counters)

  # find user John in db
  records, summary, keys = client.execute_query(
    'MATCH (u:User {name: $name}) RETURN u.name AS name',
    name='John',
    database_='memgraph',
  )
  # check result
  for record in records:
    print(record['name'])
  print(summary.query)

John
{'labels_added': 1, 'labels_removed': 0, 'nodes_created': 1, 'nodes_deleted': 0, 'properties_set': 0, 'relationships_created': 0, 'relationships_deleted': 0}
John
MATCH (u:User {name: $name}) RETURN u.name AS name


## View Data in Memgraph Lab

[Memgraph Lab](https://memgraph.com/docs/memgraph-lab) is a web-based visualization & management tool of Memgraph DBs. By default, the compose file sets up a Memgraph lab server at port `3000`. Access http://localhost:3000 to connect to the `memgraph` DB in a web browser.

In the `Query Execution` tab, run the following Cypher query from the editor to fetch all nodes from the DB and visualize them:

```cypher
MATCH (n) RETURN n
```

## Clean Up

To clean up the DB, run the following Cypher query as suggested in the [official documentation page](https://memgraph.com/docs/querying/clauses/delete#5-deleting-everything):

```cypher
MATCH (n)
DETACH DELETE n;
```

Before exiting, do not forget to stop the service container stack via `docker compose down`.